# Set up

In [912]:
import glob
import datetime as dt
import pandas as pd
import numpy as np
from datetime import timedelta
from datetime import datetime
import pytz
import pprint

import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

# Data Set Up

## Load flight data from CSV

In [251]:
# load data from The U.S. Department of Transportation's
mypath="./data/Flights-2017*.csv"
csvFileList= (glob.glob(mypath))

count=0
for f in csvFileList:
    if (count==0):
        flights=pd.read_csv(f, parse_dates= [
            'FL_DATE','DEP_TIME', 'ARR_TIME', 'CRS_DEP_TIME', 'CRS_ARR_TIME'])
    else:
        tmp=pd.read_csv(f, parse_dates= [
            'FL_DATE','DEP_TIME', 'ARR_TIME', 'CRS_DEP_TIME', 'CRS_ARR_TIME'])
        flights=flights.append(tmp, ignore_index=True, sort=False)
    count+=1
    print(count)

1
2
3
4
5
6
7
8
9
10
11
12


In [252]:
flights.dtypes

YEAR                            int64
MONTH                           int64
DAY_OF_MONTH                    int64
FL_DATE                datetime64[ns]
OP_CARRIER                     object
TAIL_NUM                       object
ORIGIN_AIRPORT_ID               int64
ORIGIN                         object
DEST_AIRPORT_ID                 int64
DEST                           object
CRS_DEP_TIME                   object
DEP_TIME                       object
DEP_DELAY                     float64
DEP_DELAY_NEW                 float64
TAXI_OUT                      float64
WHEELS_OFF                    float64
WHEELS_ON                     float64
TAXI_IN                       float64
CRS_ARR_TIME                   object
ARR_TIME                       object
ARR_DELAY                     float64
CANCELLED                     float64
DIVERTED                      float64
CRS_ELAPSED_TIME              float64
ACTUAL_ELAPSED_TIME           float64
AIR_TIME                      float64
DISTANCE    

In [8]:
airports=pd.read_csv('./data/airports.csv')

In [9]:
airports.size

2254

In [10]:
airlines=pd.read_csv('./data/airlines.csv')

In [253]:
print('Total flight rows are: '+str(flights['ORIGIN'].size))

Total flight rows are: 5666838


## Save to pickle for easy loading in future

In [254]:
flights.to_pickle('./data/flights.pkl')

In [16]:
airports.to_pickle('./data/airports.pkl')

In [17]:
airlines.to_pickle('./data/airlines.pkl')

## Load pickle files

In [3]:
flights=pd.read_pickle('./data/flights.pkl')

In [4]:
airports=pd.read_pickle('./data/airports.pkl')

In [5]:
airlines=pd.read_pickle('./data/airlines.pkl')

## Clean and merge airport and volume data

In [6]:
airport_vol=pd.read_csv('./data/airport_volume.csv')

In [7]:
airports=airports.merge(airport_vol, left_on='IATA_CODE', right_on='IATA_CODE', how='inner', left_index=True)

In [8]:
airports=airports.rename(columns={'PASS_DEPART': 'PASS_DEPART(K)'})

In [9]:
airports['PASS_DEPART(K)']=airports['PASS_DEPART(K)'].str.replace('k','')
airports['PASS_DEPART(K)']=airports['PASS_DEPART(K)'].str.replace(',','')
airports['FLIGHT_DEPART']=airports['FLIGHT_DEPART'].str.replace(',','')

In [10]:
airports['PASS_DEPART(K)']=pd.to_numeric(airports['PASS_DEPART(K)'])
airports['FLIGHT_DEPART']=pd.to_numeric(airports['FLIGHT_DEPART'])

In [11]:
airports=airports.drop(columns=['COUNTRY', 'Unnamed: 0', 'PASS_DEPART(K)'])

In [12]:
airports.dropna(subset=['LONGITUDE'], inplace=True)

In [13]:
airports.head()

,IATA_CODE,AIRPORT,CITY,STATE,LATITUDE,LONGITUDE,FLIGHT_DEPART
0,ABE,Lehigh Valley International Airport,Allentown,PA,40.65236,-75.44040,6230
1,ABI,Abilene Regional Airport,Abilene,TX,32.41132,-99.68190,3041
2,ABQ,Albuquerque International Sunport,Albuquerque,NM,35.04022,-106.60919,29424
3,ABR,Aberdeen Regional Airport,Aberdeen,SD,45.44906,-98.42183,1007
4,ABY,Southwest Georgia Regional Airport,Albany,GA,31.53552,-84.19447,1479


In [14]:
tmp=airports.loc[:,['LONGITUDE', 'LATITUDE']]

In [15]:
# Get timezone data for airport
from timezonefinder import TimezoneFinder
get_tz = TimezoneFinder().timezone_at
airports['TIMEZONE']=tmp.apply(lambda x: get_tz(lng=x['LONGITUDE'], lat=x['LATITUDE']),axis=1)

## Merge flights with airport data

In [16]:
# rows are dropped as a result of the merge, where airport codes cannot be matched
flights2=flights.merge(airports, left_on='ORIGIN', right_on='IATA_CODE', how='inner')

In [17]:
flights2.columns

Index(['YEAR', 'MONTH', 'DAY_OF_MONTH', 'FL_DATE', 'OP_CARRIER', 'TAIL_NUM',
       'ORIGIN_AIRPORT_ID', 'ORIGIN', 'DEST_AIRPORT_ID', 'DEST',
       'CRS_DEP_TIME', 'DEP_TIME', 'DEP_DELAY', 'DEP_DELAY_NEW', 'TAXI_OUT',
       'WHEELS_OFF', 'WHEELS_ON', 'TAXI_IN', 'CRS_ARR_TIME', 'ARR_TIME',
       'ARR_DELAY', 'CANCELLED', 'DIVERTED', 'CRS_ELAPSED_TIME',
       'ACTUAL_ELAPSED_TIME', 'AIR_TIME', 'DISTANCE', 'CARRIER_DELAY',
       'WEATHER_DELAY', 'NAS_DELAY', 'SECURITY_DELAY', 'LATE_AIRCRAFT_DELAY',
       'Unnamed: 32', 'IATA_CODE', 'AIRPORT', 'CITY', 'STATE', 'LATITUDE',
       'LONGITUDE', 'FLIGHT_DEPART', 'TIMEZONE'],
      dtype='object')

In [18]:
flights2=flights2[['MONTH', 'DAY_OF_MONTH', 'FL_DATE', 'OP_CARRIER', 'TAIL_NUM', 'ORIGIN', 'DEST','CRS_DEP_TIME', 'DEP_TIME', 
                   'DEP_DELAY', 'CRS_ARR_TIME', 'ARR_TIME','ARR_DELAY', 'FLIGHT_DEPART', 'TIMEZONE']]

In [19]:
flights2=flights2.rename(columns={'TIMEZONE':'ORIGIN_TIMEZONE', 'FLIGHT_DEPART':'ORIG_FLIGHTS_DEPART_VOL'})

In [20]:
print('Total flights with departure data are after cleaning: '+str(flights2['ORIGIN'].size))

Total flights with departure data are after cleaning: 5660068


## Get and clean weather data

In [23]:
weather=pd.read_pickle('./data/airport_weather.pkl')

In [24]:
# reduce rows by taking mean of records per 2-hour interval per station
tmp=weather.groupby('Station').resample('4H', on='Record_time').mean()
tmp2=weather.groupby('Station').resample('4H', on='Record_time').sum()

In [25]:
tmp['Precip']=tmp2['Precip']

In [26]:
# reset index to one level
tmp.reset_index(inplace=True)

In [27]:
# drop column that's not useful
tmp=tmp.drop(columns=['Gust_MP'])

In [28]:
# rows are dropped as a result of the merge with non-matching airports 
weather2=tmp.merge(airports, left_on='Station', right_on='IATA_CODE', 
                       how='inner')

In [29]:
weather2['TZ']=weather2['TIMEZONE'].apply(pytz.timezone)

In [30]:
weather2['TIME_UTC']=weather2.apply(lambda x: x['Record_time'].tz_localize(tz=x['TIMEZONE']).tz_convert('utc'), 
                                    axis=1)

In [31]:
station_precip=weather2.groupby('Station')['Precip'].mean()

In [32]:
station_precip=station_precip.reset_index().rename(columns={'Precip':'MEAN_PRECIP'})

In [33]:
weather2=weather2.merge(station_precip, left_on='Station', right_on='Station', 
                       how='inner')

In [34]:
weather2.to_pickle('./data/airport_weather2.pkl')

## Explore and clean flights data

In [35]:
# drop flights that did not have departure delay data (e.g. cancelled)
flights2.dropna(subset=['DEP_DELAY'], inplace=True)

In [36]:
# Fix times that are 24:00 which is not acceptible in Python
time_filter=(flights2['DEP_TIME']=='2400')
flights2.loc[time_filter,['DEP_TIME']]='0000'
flights2.loc[time_filter,['FL_DATE']]+=timedelta(days=1)

In [37]:
# convert to time for time columns
flights2['DEP_DT']=pd.to_datetime(flights2['FL_DATE'].map(str).str.slice(0,10)+' '+
                                  flights2['DEP_TIME'].map(str), format='%Y-%m-%d %H%M', errors='coerce')
flights2['ARR_DT']=pd.to_datetime(flights2['FL_DATE'].map(str).str.slice(0,10)+' '+
                                  flights2['ARR_TIME'].map(str), format='%Y-%m-%d %H%M', errors='coerce')

In [38]:
flights2.dropna(subset=['ORIGIN_TIMEZONE'], inplace=True)

In [39]:
flights2.dtypes

MONTH                               int64
DAY_OF_MONTH                        int64
FL_DATE                            object
OP_CARRIER                         object
TAIL_NUM                           object
ORIGIN                             object
DEST                               object
CRS_DEP_TIME                       object
DEP_TIME                           object
DEP_DELAY                         float64
CRS_ARR_TIME                       object
ARR_TIME                           object
ARR_DELAY                         float64
ORIG_FLIGHTS_DEPART_VOL             int64
ORIGIN_TIMEZONE                    object
DEP_DT                     datetime64[ns]
ARR_DT                     datetime64[ns]
dtype: object

In [264]:
flights2.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5580649 entries, 0 to 5660067
Data columns (total 16 columns):
MONTH                      int64
DAY_OF_MONTH               int64
FL_DATE                    object
TAIL_NUM                   object
ORIGIN                     object
DEST                       object
CRS_DEP_TIME               object
DEP_TIME                   object
DEP_DELAY                  float64
CRS_ARR_TIME               object
ARR_TIME                   object
ARR_DELAY                  float64
ORIG_FLIGHTS_DEPART_VOL    int64
ORIGIN_TIMEZONE            object
DEP_DT                     datetime64[ns]
ARR_DT                     datetime64[ns]
dtypes: datetime64[ns](2), float64(2), int64(3), object(9)
memory usage: 723.8+ MB


In [40]:
flights2['TZ']=flights2['ORIGIN_TIMEZONE'].apply(pytz.timezone)

In [41]:
# create column for departure time in UTC (note: it's key to use 'localize' to account for DST datetimes)
flights2['DEP_DT_UTC']=flights2.apply(lambda x: x['TZ'].localize(x['DEP_DT']).tz_convert('utc'), axis=1)

In [42]:
flights2=flights2.sort_values(by=['TAIL_NUM', 'DEP_DT_UTC'])

In [43]:
# create new column to account for delays due to inbound plane
flights2['INBOUND_DELAY']=flights2.groupby('TAIL_NUM')['ARR_DELAY'].apply(lambda x: x.shift(1))

In [44]:
# add new column for plane turnaround time between flights
flights2['PREV_ARR_DT']=flights2.groupby('TAIL_NUM')['ARR_DT'].shift(1)

In [45]:
flights2.dropna(subset=['INBOUND_DELAY'], inplace=True)

In [46]:
flights2.dropna(subset=['PREV_ARR_DT'], inplace=True)

In [47]:
tmp=flights2[['TZ', 'PREV_ARR_DT']]

In [48]:
flights2['PREV_ARR_DT_UTC']=tmp.apply(lambda x: x['TZ'].localize(
                                                       x['PREV_ARR_DT']).tz_convert('utc'), axis=1)

In [49]:
flights2['TURNAROUND_TIME']=flights2['DEP_DT_UTC']-flights2['PREV_ARR_DT_UTC']

In [50]:
flights2['TURNAROUND_TIME']=flights2['TURNAROUND_TIME'].dt.total_seconds()/timedelta(hours=1).total_seconds()

In [51]:
flights2[['TAIL_NUM','DEP_DT_UTC', 'CRS_DEP_TIME','DEP_TIME', 'TURNAROUND_TIME','INBOUND_DELAY', 
          "DEP_DELAY"]].head(15)

,TAIL_NUM,DEP_DT_UTC,CRS_DEP_TIME,DEP_TIME,TURNAROUND_TIME,INBOUND_DELAY,DEP_DELAY
388156,N001AA,2017-01-02 01:16:00+00:00,1850,2016,9.916667,-4.0,86.0
113741,N001AA,2017-01-02 12:55:00+00:00,0759,0755,9.700000,82.0,-4.0
1099507,N001AA,2017-01-02 15:51:00+00:00,0956,0951,1.133333,-23.0,-5.0
5279354,N001AA,2017-01-02 20:12:00+00:00,1250,1312,1.100000,-4.0,22.0
1095151,N001AA,2017-01-03 00:35:00+00:00,1835,1835,0.733333,49.0,0.0
389290,N001AA,2017-01-06 01:21:00+00:00,2010,2021,69.100000,30.0,11.0
5048951,N001AA,2017-01-06 11:31:00+00:00,0525,0531,6.900000,6.0,6.0
389394,N001AA,2017-01-08 16:17:00+00:00,1105,1117,49.033333,69.0,12.0
4100956,N001AA,2017-01-08 20:02:00+00:00,1348,1402,0.733333,18.0,14.0
391098,N001AA,2017-01-10 00:58:00+00:00,2002,1958,26.800000,-5.0,-4.0


In [52]:
flights2.columns

Index(['MONTH', 'DAY_OF_MONTH', 'FL_DATE', 'OP_CARRIER', 'TAIL_NUM', 'ORIGIN',
       'DEST', 'CRS_DEP_TIME', 'DEP_TIME', 'DEP_DELAY', 'CRS_ARR_TIME',
       'ARR_TIME', 'ARR_DELAY', 'ORIG_FLIGHTS_DEPART_VOL', 'ORIGIN_TIMEZONE',
       'DEP_DT', 'ARR_DT', 'TZ', 'DEP_DT_UTC', 'INBOUND_DELAY', 'PREV_ARR_DT',
       'PREV_ARR_DT_UTC', 'TURNAROUND_TIME'],
      dtype='object')

In [53]:
print('Total flights with departure data are: '+str(flights2['ORIGIN'].size))

Total flights with departure data are: 5558996


In [54]:
flights2.to_pickle('./data/flights2.pkl')

## Merge Flights with weather data

In [161]:
flights2=pd.read_pickle('./data/flights2.pkl')

In [55]:
weather2=pd.read_pickle('./data/airport_weather2.pkl')

In [56]:
weather2=weather2[['IATA_CODE', 'TIME_UTC', 'Temp_F', 'WindSpeed', 'Precip', 'MEAN_PRECIP']]

In [57]:
weather2=weather2.rename(columns={'IATA_CODE': 'ORIGIN'})

In [58]:
weather2.columns

Index(['ORIGIN', 'TIME_UTC', 'Temp_F', 'WindSpeed', 'Precip', 'MEAN_PRECIP'], dtype='object')

In [59]:
weather2=weather2.sort_values(by=['TIME_UTC'])

In [60]:
flights2=flights2.rename(columns={'DEP_DT_UTC':'TIME_UTC'})

In [61]:
flights2=flights2.sort_values(by=['TIME_UTC'])

In [62]:
weather2.loc[(weather2['ORIGIN']=='LAS')].head()

,ORIGIN,TIME_UTC,Temp_F,WindSpeed,Precip,MEAN_PRECIP
336336,LAS,2017-01-01 08:00:00+00:00,49.450,2.921154,0.0,0.007866
336337,LAS,2017-01-01 12:00:00+00:00,48.175,0.471154,0.0,0.007866
336338,LAS,2017-01-01 16:00:00+00:00,46.380,2.349057,0.0,0.007866
336339,LAS,2017-01-01 20:00:00+00:00,43.100,2.876923,0.0,0.007866
336340,LAS,2017-01-02 00:00:00+00:00,51.800,3.955769,0.0,0.007866


In [63]:
# merge weather data with flight data, within a 14 days tolerance date matching
flights3=pd.merge_asof(flights2, weather2, on='TIME_UTC', by='ORIGIN', tolerance=pd.Timedelta('7D'))

In [64]:
flights3.loc[(flights3['Temp_F'].isna()),['ORIGIN']].size

178692

In [65]:
flights3.columns

Index(['MONTH', 'DAY_OF_MONTH', 'FL_DATE', 'OP_CARRIER', 'TAIL_NUM', 'ORIGIN',
       'DEST', 'CRS_DEP_TIME', 'DEP_TIME', 'DEP_DELAY', 'CRS_ARR_TIME',
       'ARR_TIME', 'ARR_DELAY', 'ORIG_FLIGHTS_DEPART_VOL', 'ORIGIN_TIMEZONE',
       'DEP_DT', 'ARR_DT', 'TZ', 'TIME_UTC', 'INBOUND_DELAY', 'PREV_ARR_DT',
       'PREV_ARR_DT_UTC', 'TURNAROUND_TIME', 'Temp_F', 'WindSpeed', 'Precip',
       'MEAN_PRECIP'],
      dtype='object')

In [66]:
flights3.dropna(subset=['Temp_F','WindSpeed', 'Precip'], inplace=True)

In [67]:
flights3=flights3.rename(columns={'TIME_UTC':'DEP_DT_UTC'})

In [68]:
flights3['Precip_df']=flights3['Precip']-flights3['MEAN_PRECIP']

In [69]:
flights3.to_pickle('./data/flights3.pkl')